# Exercise 2: Monte Carlo Dice & Coins (Vectorized NumPy)

## Problem Statement
Simulate, using **only NumPy**:
1. **Game A (Dice):** Roll two fair six-sided dice. You win if the sum is 7. Estimate win probability and expected return with stake=₹1, payout=₹4 on win.
2. **Game B (Coins):** Flip 10 fair coins. You win if Heads ≥ 6. Estimate win probability and expected return with stake=₹1, payout=₹2.2 on win.
3. Compute 95% confidence intervals for the estimated probabilities using normal approximation.
4. Compare vectorized simulation vs a naive Python loop for Game A.

## Approach
- Generate large random arrays in one shot (e.g., `N=1_000_000`).
- For dice: `d1 = rng.integers(1,7, N)`, `d2 = rng.integers(1,7,N)`, `wins = (d1 + d2)==7`.
- For coins: `coins = rng.integers(0,2, (N,10))`, sum along axis=1 for heads count.
- Probability = `wins.mean()`. Expected return = `payout*p - stake`.
- CI (normal approx): `p ± 1.96 * sqrt(p*(1-p)/N)`.
- Time the vectorized vs loop approaches.

## Deliverables
- Estimated probabilities, expected returns, CIs, and timing comparison.

In [1]:
import numpy as np
from time import perf_counter

rng = np.random.default_rng(2024)
N = 100_000  # 100k trials; bump to 100_0000 if fast

# ---- Game A: Dice (vectorized) ----
t0 = perf_counter()
d1 = rng.integers(1, 7, size=N)
d2 = rng.integers(1, 7, size=N)
wins_A = (d1 + d2) == 7
p_A = wins_A.mean()
stake_A, payout_A = 1.0, 4.0
er_A = payout_A * p_A - stake_A  # expected return per play
# 95% CI for probability (normal approx)
ci_A = 1.96 * np.sqrt(p_A * (1-p_A) / N)
t1 = perf_counter()

# ---- Game B: Coins (vectorized) ----
coins = rng.integers(0, 2, size=(N, 10))  # 0=Tails, 1=Heads
heads = coins.sum(axis=1)
wins_B = heads >= 6
p_B = wins_B.mean()
stake_B, payout_B = 1.0, 2.2
er_B = payout_B * p_B - stake_B
ci_B = 1.96 * np.sqrt(p_B * (1-p_B) / N)

# ---- Game A: naive loop (slow) ----
t2 = perf_counter()
wins_loop = 0
for _ in range(N):
    a = rng.integers(1, 7)
    b = rng.integers(1, 7)
    wins_loop += int((a + b) == 7)
p_A_loop = wins_loop / N
t3 = perf_counter()

# ---- Results ----
print('Game A (Dice, sum==7)')
print(f'  p_hat: {p_A:.6f}  CI95: [{p_A-ci_A:.6f}, {p_A+ci_A:.6f}]')
print(f'  Expected return (stake=₹{stake_A}, payout=₹{payout_A}): {er_A:.4f} per play')
print(f'  Vectorized time: {(t1 - t0):.4f} s | Loop time: {(t3 - t2):.4f} s | Speedup: {(t3 - t2)/(t1 - t0 + 1e-12):.1f}x')

print('\nGame B (10 coins, win if Heads>=6)')
print(f'  p_hat: {p_B:.6f}  CI95: [{p_B-ci_B:.6f}, {p_B+ci_B:.6f}]')
print(f'  Expected return (stake=₹{stake_B}, payout=₹{payout_B}): {er_B:.4f} per play')

Game A (Dice, sum==7)
  p_hat: 0.166010  CI95: [0.163704, 0.168316]
  Expected return (stake=₹1.0, payout=₹4.0): -0.3360 per play
  Vectorized time: 0.0050 s | Loop time: 0.6175 s | Speedup: 123.8x

Game B (10 coins, win if Heads>=6)
  p_hat: 0.376630  CI95: [0.373627, 0.379633]
  Expected return (stake=₹1.0, payout=₹2.2): -0.1714 per play


Here’s a crisp, line-by-line explanation of what the Monte-Carlo code does, why it works, and the math behind each step.

---

# Imports & RNG

```python
import numpy as np
from time import perf_counter

rng = np.random.default_rng(2024)
N = 100_000  # 100k trials; bump to 100_0000 if fast
```

* `numpy` is used for **vectorized** random sampling and array math (fast, C-backed).
* `perf_counter()` is a high-resolution timer for benchmarking.
* `default_rng(2024)` seeds the generator for **reproducible** results.
* `N` = number of Monte-Carlo trials (independent plays).
  *(Minor comment typo: “100\_0000” → should be “1\_000\_000” if you want one million.)*

---

# Game A — Dice (vectorized)

```python
t0 = perf_counter()
d1 = rng.integers(1, 7, size=N)
d2 = rng.integers(1, 7, size=N)
wins_A = (d1 + d2) == 7
p_A = wins_A.mean()
stake_A, payout_A = 1.0, 4.0
er_A = payout_A * p_A - stake_A  # expected return per play
ci_A = 1.96 * np.sqrt(p_A * (1-p_A) / N)
t1 = perf_counter()
```

**What’s simulated**

* `d1`, `d2` are length-`N` arrays of iid uniform integers in `{1,…,6}` (inclusive lower bound, exclusive upper bound).
* `wins_A` is a boolean array (True/False) for each trial where the **sum equals 7**.

**Estimated probability**

* `p_A = wins_A.mean()` computes $\hat p = \frac{\text{\#wins}}{N}$ because `True`→1 and `False`→0 in NumPy.

**Expected return**

* One play stakes ₹1 and pays ₹4 **if** you win.

  $$
  \mathbb{E}[\text{return per play}] = \text{payout}\cdot \hat p - \text{stake} = 4\hat p - 1.
  $$
* Positive ER means a player-favorable game; negative means house-favorable.

**Confidence interval (normal approximation)**

* For a Bernoulli probability with estimator $\hat p$,

  $$
  \text{SE}(\hat p) = \sqrt{\frac{\hat p(1-\hat p)}{N}},\quad
  95\%\text{ CI} \approx \hat p \pm 1.96\,\text{SE}.
  $$
* `ci_A` is just the **half-width**; printing uses `p_A ± ci_A`.

**Timing**

* `t1 - t0` is the elapsed time for the **vectorized** simulation.

> Reference: The exact probability that two fair dice sum to 7 is $6/36 = 1/6 \approx 0.1667$. Your $\hat p$ will be close to that and get tighter as `N` grows.

---

# Game B — Coins (vectorized)

```python
coins = rng.integers(0, 2, size=(N, 10))  # 0=Tails, 1=Heads
heads = coins.sum(axis=1)
wins_B = heads >= 6
p_B = wins_B.mean()
stake_B, payout_B = 1.0, 2.2
er_B = payout_B * p_B - stake_B
ci_B = 1.96 * np.sqrt(p_B * (1-p_B) / N)
```

**What’s simulated**

* `coins` is an `N × 10` matrix of iid Bernoulli(0.5) draws: 0 = Tails, 1 = Heads.
* `heads` counts the 1s per row (per trial).
* Win condition: **at least 6 heads**.

**Estimated probability**

* `p_B = wins_B.mean()` estimates $\Pr(\text{Heads} \ge 6)$ for 10 fair coins.

**Expected return**

* Stake ₹1; payout ₹2.2 on win.

  $$
  \text{ER} = 2.2\,\hat p - 1.
  $$

**CI**

* Same normal approximation for the Bernoulli win indicator: `p_B ± 1.96*sqrt(p_B*(1-p_B)/N)`.

> Exact probability is $\sum_{k=6}^{10} \binom{10}{k} 2^{-10} \approx 0.376953$. Your estimate $\hat p$ should be close.

---

# Game A — Naive loop (slow baseline)

```python
t2 = perf_counter()
wins_loop = 0
for _ in range(N):
    a = rng.integers(1, 7)
    b = rng.integers(1, 7)
    wins_loop += int((a + b) == 7)
p_A_loop = wins_loop / N
t3 = perf_counter()
```

* Functionally identical to the vectorized dice simulation, but done via a **Python loop**.
* This highlights **performance**: vectorized NumPy does the same work in compiled code, avoiding Python interpreter overhead.

**Timing**

* `t3 - t2` is loop time; the printout later compares speedup `loop / vectorized`.

---

# Printing results

```python
print('Game A (Dice, sum==7)')
print(f'  p_hat: {p_A:.6f}  CI95: [{p_A-ci_A:.6f}, {p_A+ci_A:.6f}]')
print(f'  Expected return (stake=₹{stake_A}, payout=₹{payout_A}): {er_A:.4f} per play')
print(f'  Vectorized time: {(t1 - t0):.4f} s | Loop time: {(t3 - t2):.4f} s | Speedup: {(t3 - t2)/(t1 - t0 + 1e-12):.1f}x')

print('\nGame B (10 coins, win if Heads>=6)')
print(f'  p_hat: {p_B:.6f}  CI95: [{p_B-ci_B:.6f}, {p_B+ci_B:.6f}]')
print(f'  Expected return (stake=₹{stake_B}, payout=₹{payout_B}): {er_B:.4f} per play')
```

* Shows the estimated probabilities $\hat p$, their **95% CIs**, **expected returns**, and the **speedup** of vectorization.
* The tiny `+ 1e-12` prevents division by zero if the vectorized section is extremely fast.

---

## Why vectorization wins

* The vectorized paths create large arrays (`size=N`) in one go and do bulk operations in **compiled C** (via NumPy), which is typically **tens to hundreds of times faster** than looping in Python.
* Both methods are $O(N)$, but vectorization drastically reduces Python overhead and benefits from SIMD/cache-friendly memory access.

## Assumptions & notes

* Dice and coin flips are modeled as **independent and fair**.
* Normal CI is an approximation; with large `N` it’s excellent for Bernoulli proportions.
* If you push `N` to millions, consider memory limits; you can simulate in **chunks** and aggregate (same math, lower peak memory).

